VVVV Imports VVVVV

In [1]:
import subprocess
import sys
import os

try:
  import numpy as np
except:
  subprocess.run([sys.executable,'-m','pip', 'install','numpy'])
  import numpy as np
print(f"numpy: {np.__version__}")

try:
  import torch
except:
  subprocess.run([sys.executable,'-m','pip', 'install','torch'])
  import torch
print(f"torch version: {torch.__version__}")
from torch import nn
from torch.utils.data import DataLoader

try:
  from pathlib import Path
except:
  subprocess.run([sys.executable,'-m','pip', 'install','pathlib'])
  from pathlib import Path
print(f"pathlib check: {Path}")

try:
  import requests
except:
  subprocess.run([sys.executable,'-m','pip', 'install','requests'])
  import requests
print(f"requests version: {requests.__version__}")

try:
  import zipfile
except:
  subprocess.run([sys.executable,'-m','pip','install','zipfile'])
  import zipfile
print(f"zipfile check {zipfile}")

try:
  from tqdm.auto import tqdm
except:
  subprocess.run([sys.executable,'-m','pip','install','tdqm'])
  from tqdm.auto import tqdm
print(f"tqdm check {tqdm}")


try:
  import torchvision
except:
  subprocess.run([sys.executable,'-m','pip','install','torchvision'])
  import torchvision
print(f"torchvision version:{torchvision.__version__}")
from torchvision import datasets, transforms

numpy: 2.0.2
torch version: 2.6.0+cu124
pathlib check: <class 'pathlib.Path'>
requests version: 2.32.3
zipfile check <module 'zipfile' from '/usr/lib/python3.11/zipfile.py'>
tqdm check <class 'tqdm.auto.tqdm'>
torchvision version:0.21.0+cu124


Our models are underperforming (not fitting the data well). What are 3 methods for preventing underfitting? Write them down and explain each with a sentence.



*   Add more to your model via hidden units or layers.</br>
This increase the number of parameters that a model can learn.
*   Adjust the learning rate</br>
a learning rate that is too high can result in a model adjusting the weights by too much over shooting the sweet spot.
* More epochs. </br>
The more time it trains the more it can pursue that sweet spot. Each epoch gets us closer unless we are over shooting so a few more won't hurt.



Recreate the data loading functions we built in sections 1, 2, 3 and 4. You should have train and test DataLoader's ready to use.


In [2]:
data_path = Path("/data")
image_path= data_path / "pizza_steak_sushi"
if data_path.is_dir():
  print("data_path already exists")
else:
  print("making data_path")
  data_path.mkdir(parents=True, exist_ok=True)

if image_path.is_dir():
  print("image_path already exists")
else:
  print("making image_path")
  image_path.mkdir(parents=True, exist_ok=True)

zip_file_path = data_path/"pizza_steak_sushi.zip"

with open(zip_file_path,"wb") as f:
  request_for_zip = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip")
  print("Downloading zip file")
  f.write(request_for_zip.content)

  with zipfile.ZipFile(zip_file_path,"r") as zipperman:
    print("Extracting")
    zipperman.extractall(path=image_path)

making data_path
making image_path
Extracting


In [3]:
image_path_list = list(image_path.glob("*/*/*.jpg"))
train_dir = image_path/"train"
test_dir = image_path/"test"
input_transform = transforms.Compose([transforms.Resize(size=(64,64)),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor()
                                      ])
print(train_dir)
train_data = datasets.ImageFolder(root=train_dir,
                                  transform=input_transform)
test_data = datasets.ImageFolder(root=test_dir,
                                 transform=input_transform)
print(f"\nTrain data: {train_data}")
print(f"\nTest data: {test_data}")

BATCH_SIZE = 1
NUMBER_OF_WORKERS = os.cpu_count()

train_dataloaders = DataLoader(dataset=train_data,
                               batch_size=BATCH_SIZE,
                               shuffle=True,
                               num_workers=NUMBER_OF_WORKERS)

test_dataloaders = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=NUMBER_OF_WORKERS)

/data/pizza_steak_sushi/train

Train data: Dataset ImageFolder
    Number of datapoints: 225
    Root location: /data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )

Test data: Dataset ImageFolder
    Number of datapoints: 75
    Root location: /data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


Recreate model_0 we built in section 7.

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
class Tiny_VGG(nn.Module):
  def __init__(self, input_shape,hidden_units, output_shape):
    super().__init__()
    self.convo_block_01 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.MaxPool2d(kernel_size=2)
    )

    self.convo_block_02 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13, #So this needs fix later.
                  out_features=output_shape
                  )
    )
  def forward(self, x, print_shapes = False):
    x = self.convo_block_01(x)
    if(print_shapes):
      print(x.shape)
    x = self.convo_block_02(x)
    if(print_shapes):
      print(x.shape)
    x = self.classifier(x)
    if(print_shapes):
      print(x.shape)
    return x



In [6]:
class_names = train_data.classes
model_0 = Tiny_VGG(input_shape=3,
                   hidden_units=10,
                   output_shape=len(class_names))
img_samp, label_samp = next(iter(train_data))
print(f"Sameple shape: {img_samp.unsqueeze(0).shape}")
model_0(img_samp.unsqueeze(0),print_shapes=True)

Sameple shape: torch.Size([1, 3, 64, 64])
torch.Size([1, 10, 30, 30])
torch.Size([1, 10, 13, 13])
torch.Size([1, 3])


tensor([[ 0.0048,  0.0107, -0.0525]], grad_fn=<AddmmBackward0>)

In [7]:
def training_step(model:torch.nn.Module,
                  data_loader: torch.utils.data.DataLoader,
                  loss_fn:torch.nn.Module,
                  optimizer:torch.optim.Optimizer,
                  device = "cuda" if torch.cuda.is_available else "cpu"):
  train_acc, train_loss = 0 , 0
  model.train
  for batch, (x,y) in enumerate(data_loader):
    x,y = x.to(device), y.to(device)
    logits = model(x)
    loss = loss_fn(logits, y)
    train_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pred_class = torch.argmax(torch.softmax(logits,dim=1),dim=1)
    train_acc += (pred_class==y).sum().item()/len(logits)
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  return train_loss, train_acc



In [8]:
def test_step(model:torch.nn.Module, data_loader, loss_fn, device):
  test_acc, test_loss = 0,0
  model.eval()
  with torch.inference_mode():
    for batch, (x,y) in enumerate(data_loader):
      x, y = x.to(device), y.to(device)
      logits = model(x)
      loss = loss_fn(logits, y)
      test_loss += loss.item()
      test_preds = torch.argmax(torch.softmax(logits, dim=1), dim=1)
      test_acc +=(test_preds == y).sum().item()/len(test_preds)
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    return test_loss, test_acc


In [9]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device = "cuda" if torch.cuda.is_available() else "cpu"):
  results = {"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[]}
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = training_step(model=model,
                                          data_loader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
    test_loss, test_acc = test_step(model=model,
                                    data_loader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)
    print(f"train_loss: {train_loss:.4f}, train_acc:{train_acc:.2f}, test_loss {test_loss:.4f},test_acc{test_acc:.2f}")
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results

In [10]:
model_0 = Tiny_VGG(input_shape=3,
                   hidden_units=10,
                   output_shape=len(class_names))

optimizer = torch.optim.Adam(model_0.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()
ResultsOf5Epochs = train(model=model_0,
                         train_dataloader=train_dataloaders,
                         test_dataloader=test_dataloaders,
                         optimizer=optimizer,
                         loss_fn=loss_fn,
                         epochs=5,
                         device=device)
print(ResultsOf5Epochs)

model_0 = Tiny_VGG(input_shape=3,
                   hidden_units=10,
                   output_shape=len(class_names))

optimizer = torch.optim.Adam(model_0.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()
ResultsOf20Epochs = train(model=model_0,
                         train_dataloader=train_dataloaders,
                         test_dataloader=test_dataloaders,
                         optimizer=optimizer,
                         loss_fn=loss_fn,
                         epochs=20,
                         device=device)
print(ResultsOf20Epochs)

model_0 = Tiny_VGG(input_shape=3,
                   hidden_units=10,
                   output_shape=len(class_names))

optimizer = torch.optim.Adam(model_0.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()
ResultsOf50Epochs = train(model=model_0,
                         train_dataloader=train_dataloaders,
                         test_dataloader=test_dataloaders,
                         optimizer=optimizer,
                         loss_fn=loss_fn,
                         epochs=50,
                         device=device)
print(ResultsOf50Epochs)

  0%|          | 0/5 [00:00<?, ?it/s]

train_loss: 1.0293, train_acc:0.45, test_loss 1.0070,test_acc0.45
train_loss: 0.9071, train_acc:0.57, test_loss 0.9739,test_acc0.53
train_loss: 0.9116, train_acc:0.56, test_loss 0.9948,test_acc0.43
train_loss: 0.8356, train_acc:0.63, test_loss 0.9934,test_acc0.45
train_loss: 0.7990, train_acc:0.65, test_loss 1.0037,test_acc0.45
{'train_loss': [tensor(1.0293, grad_fn=<DivBackward0>), tensor(0.9071, grad_fn=<DivBackward0>), tensor(0.9116, grad_fn=<DivBackward0>), tensor(0.8356, grad_fn=<DivBackward0>), tensor(0.7990, grad_fn=<DivBackward0>)], 'train_acc': [0.4533333333333333, 0.5688888888888889, 0.56, 0.6266666666666667, 0.6533333333333333], 'test_loss': [1.007024623254935, 0.9738552349805832, 0.994814891765515, 0.9934147020460417, 1.003729272208487], 'test_acc': [0.4533333333333333, 0.5333333333333333, 0.4266666666666667, 0.4533333333333333, 0.4533333333333333]}


  0%|          | 0/20 [00:00<?, ?it/s]

train_loss: 1.0837, train_acc:0.44, test_loss 0.9992,test_acc0.35
train_loss: 0.9005, train_acc:0.57, test_loss 0.9726,test_acc0.45
train_loss: 0.8550, train_acc:0.58, test_loss 0.9346,test_acc0.48
train_loss: 0.8066, train_acc:0.63, test_loss 1.0048,test_acc0.45
train_loss: 0.8151, train_acc:0.68, test_loss 1.0557,test_acc0.43
train_loss: 0.7226, train_acc:0.65, test_loss 1.0758,test_acc0.55
train_loss: 0.6936, train_acc:0.69, test_loss 1.0364,test_acc0.52
train_loss: 0.6394, train_acc:0.76, test_loss 1.1301,test_acc0.45
train_loss: 0.5896, train_acc:0.74, test_loss 1.0910,test_acc0.55
train_loss: 0.5587, train_acc:0.73, test_loss 1.0950,test_acc0.56
train_loss: 0.4929, train_acc:0.81, test_loss 1.0488,test_acc0.52
train_loss: 0.4812, train_acc:0.79, test_loss 1.1627,test_acc0.47
train_loss: 0.3708, train_acc:0.87, test_loss 1.2592,test_acc0.53
train_loss: 0.3807, train_acc:0.87, test_loss 1.1755,test_acc0.51
train_loss: 0.3442, train_acc:0.86, test_loss 1.4508,test_acc0.45
train_loss

  0%|          | 0/50 [00:00<?, ?it/s]

train_loss: 1.1129, train_acc:0.32, test_loss 1.0817,test_acc0.44
train_loss: 1.0809, train_acc:0.41, test_loss 0.9843,test_acc0.47
train_loss: 0.9830, train_acc:0.47, test_loss 1.0050,test_acc0.47
train_loss: 0.9201, train_acc:0.60, test_loss 1.0372,test_acc0.44
train_loss: 0.9312, train_acc:0.57, test_loss 1.0183,test_acc0.49
train_loss: 0.8576, train_acc:0.61, test_loss 1.0206,test_acc0.47
train_loss: 0.7992, train_acc:0.68, test_loss 1.0891,test_acc0.47
train_loss: 0.8138, train_acc:0.67, test_loss 1.3674,test_acc0.37
train_loss: 0.7627, train_acc:0.68, test_loss 1.2389,test_acc0.53
train_loss: 0.7241, train_acc:0.68, test_loss 1.2591,test_acc0.49
train_loss: 0.7082, train_acc:0.69, test_loss 1.0361,test_acc0.55
train_loss: 0.6092, train_acc:0.75, test_loss 1.1873,test_acc0.53
train_loss: 0.6010, train_acc:0.76, test_loss 1.1395,test_acc0.52
train_loss: 0.5372, train_acc:0.78, test_loss 1.1411,test_acc0.51
train_loss: 0.4568, train_acc:0.83, test_loss 1.1648,test_acc0.53
train_loss

In [11]:
model_1 = Tiny_VGG(input_shape=3,
                   hidden_units=20,
                   output_shape=len(class_names))

optimizer = torch.optim.Adam(model_1.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()
ResultsOf20EpochsAfterDoubleHiddenUnits = train(model=model_1,
                         train_dataloader=train_dataloaders,
                         test_dataloader=test_dataloaders,
                         optimizer=optimizer,
                         loss_fn=loss_fn,
                         epochs=20,
                         device=device)
print(ResultsOf20EpochsAfterDoubleHiddenUnits)

  0%|          | 0/20 [00:00<?, ?it/s]

train_loss: 1.1074, train_acc:0.42, test_loss 1.0006,test_acc0.45
train_loss: 1.0459, train_acc:0.49, test_loss 1.2784,test_acc0.45
train_loss: 0.9382, train_acc:0.54, test_loss 1.0131,test_acc0.44
train_loss: 0.9208, train_acc:0.61, test_loss 1.0441,test_acc0.43
train_loss: 0.8292, train_acc:0.64, test_loss 1.0169,test_acc0.44
train_loss: 0.8391, train_acc:0.68, test_loss 1.0697,test_acc0.43
train_loss: 0.7972, train_acc:0.66, test_loss 1.1006,test_acc0.48
train_loss: 0.6992, train_acc:0.68, test_loss 1.1295,test_acc0.41
train_loss: 0.6963, train_acc:0.72, test_loss 1.0376,test_acc0.55
train_loss: 0.6816, train_acc:0.70, test_loss 1.2214,test_acc0.49
train_loss: 0.6664, train_acc:0.72, test_loss 1.6962,test_acc0.49
train_loss: 0.5177, train_acc:0.80, test_loss 1.3568,test_acc0.55
train_loss: 0.4288, train_acc:0.83, test_loss 1.3624,test_acc0.53
train_loss: 0.3727, train_acc:0.88, test_loss 1.9086,test_acc0.51
train_loss: 0.4257, train_acc:0.85, test_loss 1.3629,test_acc0.56
train_loss

In [12]:
with open(zip_file_path,"wb") as f:
  request_for_zip = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi_20_percent.zip")
  print("Downloading zip file")
  f.write(request_for_zip.content)

  with zipfile.ZipFile(zip_file_path,"r") as zipperman:
    print("Extracting")
    zipperman.extractall(path=image_path)

image_path_list = list(image_path.glob("*/*/*.jpg"))
train_dir = image_path/"train"
test_dir = image_path/"test"
input_transform = transforms.Compose([transforms.Resize(size=(64,64)),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor()
                                      ])
print(train_dir)
train_data = datasets.ImageFolder(root=train_dir,
                                  transform=input_transform)
test_data = datasets.ImageFolder(root=test_dir,
                                 transform=input_transform)
print(f"\nTrain data: {train_data}")
print(f"\nTest data: {test_data}")

BATCH_SIZE = 1
NUMBER_OF_WORKERS = os.cpu_count()

train_dataloaders = DataLoader(dataset=train_data,
                               batch_size=BATCH_SIZE,
                               shuffle=True,
                               num_workers=NUMBER_OF_WORKERS)

test_dataloaders = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=NUMBER_OF_WORKERS)

Extracting
/data/pizza_steak_sushi/train

Train data: Dataset ImageFolder
    Number of datapoints: 480
    Root location: /data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )

Test data: Dataset ImageFolder
    Number of datapoints: 211
    Root location: /data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [13]:
model_2 = Tiny_VGG(input_shape=3,
                   hidden_units=20,
                   output_shape=len(class_names))

optimizer = torch.optim.Adam(model_1.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()
ResultsDoubleDataset = train(model=model_1,
                         train_dataloader=train_dataloaders,
                         test_dataloader=test_dataloaders,
                         optimizer=optimizer,
                         loss_fn=loss_fn,
                         epochs=20,
                         device=device)
print(ResultsDoubleDataset)

  0%|          | 0/20 [00:00<?, ?it/s]

train_loss: 0.8375, train_acc:0.70, test_loss 1.2311,test_acc0.55
train_loss: 0.6565, train_acc:0.76, test_loss 1.1800,test_acc0.60
train_loss: 0.5878, train_acc:0.77, test_loss 1.2355,test_acc0.59
train_loss: 0.5044, train_acc:0.81, test_loss 1.3921,test_acc0.61
train_loss: 0.4871, train_acc:0.79, test_loss 2.1457,test_acc0.56
train_loss: 0.4696, train_acc:0.81, test_loss 1.6032,test_acc0.53
train_loss: 0.3982, train_acc:0.85, test_loss 1.5256,test_acc0.52
train_loss: 0.3463, train_acc:0.86, test_loss 1.5134,test_acc0.55
train_loss: 0.2743, train_acc:0.90, test_loss 1.9385,test_acc0.55
train_loss: 0.2456, train_acc:0.91, test_loss 2.1640,test_acc0.54
train_loss: 0.2111, train_acc:0.92, test_loss 2.5572,test_acc0.49
train_loss: 0.3341, train_acc:0.91, test_loss 2.3188,test_acc0.52
train_loss: 0.2377, train_acc:0.92, test_loss 2.4341,test_acc0.55
train_loss: 0.1401, train_acc:0.95, test_loss 2.6931,test_acc0.55
train_loss: 0.1595, train_acc:0.95, test_loss 2.8392,test_acc0.54
train_loss

In [14]:
custom_image_path = data_path/"test_pizza.jpeg"
#https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTVJiMGt-SI69kdqasJ_evpDrAnV_NmCvgkMw&s
if custom_image_path.is_file()== False:
  with open(custom_image_path,"wb") as f:
    custom_image_request = requests.get("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTVJiMGt-SI69kdqasJ_evpDrAnV_NmCvgkMw&s")
    print("Downloading custom image")
    f.write(custom_image_request.content)
else:
  print("This image already exists")


In [18]:
custom_image = torchvision.io.read_image(str(custom_image_path)).unsqueeze(0).type(torch.float32).to(device=device)
Image_transform = transforms.Compose([transforms.Resize(size=(64,64))])
custom_image = Image_transform(custom_image)
model_1.eval()
with torch.inference_mode():
  pred = model_1(custom_image)
  custom_image_label = torch.argmax(torch.softmax(pred, dim=1),dim=1)
  print(f"Identified class: {class_names[custom_image_label]}")

Identified class: pizza
